In [1]:

from __future__ import division
import numpy as np
import scipy.linalg
import bempp.api as bem
import scipy 

maxiter = 10000
restart = 10000
tolerance =  1e-08

kExt = 1.
kInt = 2.
rhoExt = 1
rhoInt = 1
R = 1

alpha = rhoInt / rhoExt

l_max = 200
l = np.arange(0, l_max + 1)


def rescale(A, alpha):
    """Rescale the 2x2 block operator matrix A"""
    A[0, 1] = A[0, 1] * alpha
    A[1, 0] = A[1, 0] * (1/alpha)
    return A

bem.global_parameters.hmat.max_block_size = 1000


def dirichlet_fun(x, n, domain_index, result):
    result[0] = np.exp(1j * kExt * x[0])

def neumann_fun(x, n, domain_index, result):
    result[0] = 1j * kExt * np.exp(1j * kExt * x[0]) * n[0]

from scipy.sparse.linalg import LinearOperator
class DenseTensorLinearOperator(LinearOperator):
    
    def __init__(self, A0, A1):
        self.A0 = A0
        self.A1 = A1
        self.n0 = self.A0.shape[0]
        self.n1 = self.A1.shape[0]
        self.shape = self.n0*self.n1, self.n0*self.n1
        self.dtype = self.A0.dtype

    def _matvec(self, x):
        n0 = self.A0.shape[0]
        n1 = self.A1.shape[0]
        xbis0 = x.reshape([n0, n1])
        ytemp = self.A0 * xbis0
        ztemp = (self.A1 * ytemp.T).T.reshape(n0*n1)
        return ztemp

#bem.global_parameters.assembly.boundary_operator_assembly_type = 'dense'
print bem.global_parameters.assembly.boundary_operator_assembly_type

nQuad = 3
bem.global_parameters.quadrature.near.single_order = nQuad+2
bem.global_parameters.quadrature.near.double_order = nQuad+2
bem.global_parameters.quadrature.medium.single_order = nQuad+1
bem.global_parameters.quadrature.medium.double_order = nQuad+1
bem.global_parameters.quadrature.far.single_order = nQuad
bem.global_parameters.quadrature.far.double_order = nQuad



from projection import refine, discrete_coefficients_projection


h0 = 1
h1 = 0.8

grid = bem.shapes.sphere(h=h0)
space = bem.function_space(grid, "P", 1)
AExt = bem.operators.boundary.helmholtz.multitrace_operator(grid, kExt)
I  = bem.operators.boundary.sparse.multitrace_identity(grid, spaces='linear')
AInt = bem.operators.boundary.helmholtz.multitrace_operator(grid, kInt)
STF = rescale(AInt, alpha) + AExt
dirichlet_grid_fun = bem.GridFunction(space, fun=dirichlet_fun)
neumann_grid_fun = bem.GridFunction(space, fun=neumann_fun)
rhs = [dirichlet_grid_fun, neumann_grid_fun]
rhs = np.concatenate([(STF * rhs )[0].coefficients,(STF * rhs)[1].coefficients], axis=0)

rhs0 = rhs
A0 = STF.strong_form()

grid = bem.shapes.sphere(h=h1)
space = bem.function_space(grid, "P", 1)
AExt = bem.operators.boundary.helmholtz.multitrace_operator(grid, kExt)
I  = bem.operators.boundary.sparse.multitrace_identity(grid, spaces='linear')
AInt = bem.operators.boundary.helmholtz.multitrace_operator(grid, kInt)
STF = rescale(AInt, alpha) + AExt
dirichlet_grid_fun = bem.GridFunction(space, fun=dirichlet_fun)
neumann_grid_fun = bem.GridFunction(space, fun=neumann_fun)
rhs = [dirichlet_grid_fun, neumann_grid_fun]
rhs = np.concatenate([(STF * rhs )[0].coefficients,(STF * rhs)[1].coefficients], axis=0)
rhs1 = rhs
A1 = STF.strong_form()

A = DenseTensorLinearOperator(A0, A1)
rhs= np.dot(np.array([rhs0]).T, np.array([rhs1]))

import time 
t0 = time.time()
from login import gmres
x, info, res = gmres( A ,rhs.ravel(), tol=tolerance, return_residuals=True,  maxiter=maxiter, restart=restart)
t0 = time.time()-t0


hmat


/usr/lib/python2.7/dist-packages/scipy/sparse/linalg/dsolve/linsolve.py:243: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


iteration - 1 || residual - 0.895181020496
iteration - 2 || residual - 0.278984934911
iteration - 3 || residual - 0.260424066561
iteration - 4 || residual - 0.0816753954977
iteration - 5 || residual - 0.0616469428373
iteration - 6 || residual - 0.0122587196963
iteration - 7 || residual - 0.010386846177
iteration - 8 || residual - 0.00336986382098
iteration - 9 || residual - 0.00274378400231
iteration - 10 || residual - 0.000322985193764
iteration - 11 || residual - 0.000270003092911
iteration - 12 || residual - 6.14807065583e-05
iteration - 13 || residual - 5.57876069973e-05
iteration - 14 || residual - 6.39093812473e-06
iteration - 15 || residual - 6.25070110241e-06
iteration - 16 || residual - 7.52423833396e-07
iteration - 17 || residual - 7.35952044771e-07
iteration - 18 || residual - 2.00199940728e-07
iteration - 19 || residual - 1.89441282166e-07
iteration - 20 || residual - 1.50010629032e-08
iteration - 21 || residual - 1.30857194878e-08
iteration - 22 || residual - 2.27620618652

In [2]:
A.n0,A.n1

(52, 52)

In [3]:
rhs.shape

(52, 52)